In [2]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
import math
import time
from numba import jit
from mpl_toolkits.mplot3d import Axes3D


from matplotlib.colors import LinearSegmentedColormap
import functools
import itertools as IT
import scipy.ndimage as ndimage
import matplotlib.colors as mcolors
import matplotlib.cm as cm

In [100]:
@jit(nopython = True,nogil = True,fastmath= True)
def Laplacian_m(L,A,deltax):

    #CFL condition delta x => sqrt(D) delta t, where D is the number of dimensions            
    h  = L.shape[0]
    for i in range(0, h):
        for j in range(0,h):
            for k in range(0,h):
                L[i,j,k] = (A[(i-1)%h,(j-1)%h,(k-1)%h] + 3*A[(i-1)%h,j,(k-1)%h] + A[(i-1)%h,(j+1)%h,(k-1)%h] \
                + 3*A[(i-1)%h,(j-1)%h,k] + 14*A[(i-1)%h,j,k] + 3*A[(i-1)%h,(j+1)%h,k] \
                + A[(i-1)%h,(j-1)%h,(k+1)%h] + 3*A[(i-1)%h,j,(k+1)%h] + A[(i-1)%h,(j+1)%h,(k+1)%h] \
                + 3*A[i,(j-1)%h,(k-1)%h] + 14*A[i,j,(k-1)%h] + 3*A[i,(j+1)%h,(k-1)%h] \
                + 14*A[i,(j-1)%h,k] - 128*A[i,j,k] + 14*A[i,(j+1)%h,k] \
                + 3*A[i,(j-1)%h,(k+1)%h] + 14*A[i,j,(k+1)%h] + 3*A[i,(j+1)%h,(k+1)%h] \
                + A[(i+1)%h,(j-1)%h,(k-1)%h] + 3*A[(i+1)%h,j,(k-1)%h] + A[(i+1)%h,(j+1)%h,(k-1)%h] \
                + 3*A[(i+1)%h,(j-1)%h,k] + 14*A[(i+1)%h,j,k] + 3*A[(i+1)%h,(j+1)%h,k] \
                + A[(i+1)%h,(j-1)%h,(k+1)%h] + 3*A[(i+1)%h,j,(k+1)%h] + A[(i+1)%h,(j+1)%h,(k+1)%h])/(30*pow(deltax,2))

                

def update_m(A1,A2,vel1,vel2,M1,M2,omega,eta,deltax,deltat,time):

    a = 4.36000000000006e-18*time - 6.78288102293483e-23
    beta = 0
    alpha = 3
    era = 1 #radiation

    lam = 2*pow(math.pi,2)/pow(omega,2)
    

    F1 = M1 - a**beta * lam*A1*(pow(A1,2)+pow(A2,2)-pow(eta,2))-alpha*era*vel1
    F2 = M2 - a**beta * lam*A2*(pow(A1,2)+pow(A2,2)-pow(eta,2))-alpha*era*vel2
    
    a1 = 4.36000000000006e-18*(time + deltat) - 6.78288102293483e-23
    
    A1 += deltat*(vel1 + 0.5*deltat*F1) 
    A2 += deltat*(vel2 + 0.5*deltat*F2) 
    
    Laplacian_m(M1,A1,deltax)
    Laplacian_m(M2,A2,deltax)
    
    x1 = -alpha*era*vel1
    x2 = -alpha*era*vel2

    
    vel1 +=  deltat/2*(F1 + M1 - a1**beta *lam*A1*(pow(A1,2)+pow(A2,2)-pow(eta,2))  + x1) 
    vel2 +=  deltat/2*(F2 + M2 - a1**beta *lam*A2*(pow(A1,2)+pow(A2,2)-pow(eta,2))  + x2)
    
    

def plotting_m(N,t0,t,A1,A2,vel1,vel2,omega,eta,deltax,deltat):

    time = t0
    

#     fig = plt.figure(figsize=(8,8))
#     #fig.set_facecolor('springgreen')
#     ax = fig.add_subplot(1, 1, 1, projection='3d')
#     ax.set_facecolor('springgreen')
#     #ax.set_axis_off()

#     ax.set_xticks([])
#     ax.set_yticks([])
#     ax.set_zticks([])
#     ax.set_xlim3d(0,N)
#     ax.set_ylim3d(0,N)
#     ax.set_zlim3d(0,N)
#     m = 5

 
    #angle = np.arctan2(A2,A1)
    
    M1 = np.zeros((N,N,N))
    M2 = np.zeros((N,N,N))
    
    Laplacian_m(M1,A1,deltax)
    Laplacian_m(M2,A2,deltax)

    for i in np.arange(int(t/deltat)):
        time = round(time,2)
        
                    
     
#         if time <= m*5: 
#             if time % 5 == 0:
#                 mod = np.where(np.sqrt(pow(A1,2)+pow(A2,2))**1 < 0.5)

#                 scat = ax.scatter(mod[0],mod[1],mod[2],color= 'black')

#                 plt.savefig('plotting_m/'+str(int(time/5)).zfill(3)+'.png')

#                 scat.remove()
                    
#         else:        
        
#             if time % 5 == 0:

#                 mod = np.where(np.sqrt(pow(A1,2)+pow(A2,2))**1 < 0.5)

#                 scat = ax.scatter(mod[0],mod[1],mod[2],color= 'black')

#                 plt.savefig('plotting_m/'+str(int(time-m*4)).zfill(3)+'.png')

#                 scat.remove()
            
        update_m(A1,A2,vel1,vel2,M1,M2,omega,eta,deltax,deltat,time)
        time = time + deltat

In [101]:
@jit(nopython = True,nogil = True,fastmath= True)
def Laplacian(A,deltax):

    h = len(A)
    #CFL condition delta x => sqrt(D) delta t, where D is the number of dimensions            
    P = np.zeros((h,h,h))
  
    for i in range(0, h):
        for j in range(0,h):
            for k in range(0,h):
                P[i,j,k] = (A[(i-1)%h,(j-1)%h,(k-1)%h] + 3*A[(i-1)%h,j,(k-1)%h] + A[(i-1)%h,(j+1)%h,(k-1)%h] \
                + 3*A[(i-1)%h,(j-1)%h,k] + 14*A[(i-1)%h,j,k] + 3*A[(i-1)%h,(j+1)%h,k] \
                + A[(i-1)%h,(j-1)%h,(k+1)%h] + 3*A[(i-1)%h,j,(k+1)%h] + A[(i-1)%h,(j+1)%h,(k+1)%h] \
                + 3*A[i,(j-1)%h,(k-1)%h] + 14*A[i,j,(k-1)%h] + 3*A[i,(j+1)%h,(k-1)%h] \
                + 14*A[i,(j-1)%h,k] - 128*A[i,j,k] + 14*A[i,(j+1)%h,k] \
                + 3*A[i,(j-1)%h,(k+1)%h] + 14*A[i,j,(k+1)%h] + 3*A[i,(j+1)%h,(k+1)%h] \
                + A[(i+1)%h,(j-1)%h,(k-1)%h] + 3*A[(i+1)%h,j,(k-1)%h] + A[(i+1)%h,(j+1)%h,(k-1)%h] \
                + 3*A[(i+1)%h,(j-1)%h,k] + 14*A[(i+1)%h,j,k] + 3*A[(i+1)%h,(j+1)%h,k] \
                + A[(i+1)%h,(j-1)%h,(k+1)%h] + 3*A[(i+1)%h,j,(k+1)%h] + A[(i+1)%h,(j+1)%h,(k+1)%h])/(30*pow(deltax,2))
    return P 




def update(A1,A2,vel1,vel2,M1,M2,omega,eta,deltax,deltat,time):
    
    a = 4.36000000000006e-18*time - 6.78288102293483e-23
    beta = 0
    alpha = 3
    era = 1 #radiation

    lam = 2*pow(math.pi,2)/pow(omega,2)
    
    F1 = M1 - a**beta * lam*A1*(pow(A1,2)+pow(A2,2)-pow(eta,2))-alpha*era*vel1
    F2 = M2 - a**beta * lam*A2*(pow(A1,2)+pow(A2,2)-pow(eta,2))-alpha*era*vel2
    
    time = time + deltat
    
    a1 = 4.36000000000006e-18*time - 6.78288102293483e-23
    
    A1 = A1 + deltat*(vel1 + 0.5*deltat*F1) 
    A2 = A2 + deltat*(vel2 + 0.5*deltat*F2) 
    
    L1 = Laplacian(A1,deltax)
    L2 = Laplacian(A2,deltax)

    
    vel1 = vel1 + deltat/2*(F1 + L1 - a1**beta *lam*A1*(pow(A1,2)+pow(A2,2)-pow(eta,2)) -alpha*era*vel1) 
    vel2 = vel2 + deltat/2*(F2 + L2 - a1**beta *lam*A2*(pow(A1,2)+pow(A2,2)-pow(eta,2)) -alpha*era*vel2)
    
    
    return A1,A2,vel1,vel2,L1,L2,time


#change the values accordlingly    
def plotting(N,t0,t,A1,A2,vel1,vel2,omega,eta,deltax,deltat):

    time = t0

#     fig = plt.figure(figsize=(8,8))
#     #fig.set_facecolor('springgreen')
#     ax = fig.add_subplot(1, 1, 1, projection='3d')
#     ax.set_facecolor('springgreen')
#     #ax.set_axis_off()

#     ax.set_xticks([])
#     ax.set_yticks([])
#     ax.set_zticks([])
#     ax.set_xlim3d(0,N)
#     ax.set_ylim3d(0,N)
#     ax.set_zlim3d(0,N)
#     m = 5

 
    #angle = np.arctan2(A2,A1)
    
    M1 = Laplacian(A1,deltax)
    M2 = Laplacian(A2,deltax)


    for i in np.arange(int(t/deltat)):
        time = round(time,2)
        
                 
     
#         if time <= m*5:
#             if time % 5 == 0:
#                 mod = np.where(np.sqrt(pow(A1,2)+pow(A2,2))**1 < 0.5)

#                 scat = ax.scatter(mod[0],mod[1],mod[2],color= 'black')

#                 plt.savefig('plotting/'+str(int(time/5)).zfill(3)+'.png')

#                 scat.remove()
                    
#         else:        
        
#             if time % 5 == 0:

#                 mod = np.where(np.sqrt(pow(A1,2)+pow(A2,2))**1 < 0.5)

#                 scat = ax.scatter(mod[0],mod[1],mod[2],color= 'black')

#                 plt.savefig('plotting/'+str(int(time-m*4)).zfill(3)+'.png')

#                 scat.remove()
            
        A1,A2,vel1,vel2,M1,M2,time = update(A1,A2,vel1,vel2,M1,M2,omega,eta,deltax,deltat,time)




In [102]:
n = 7;
deltax = 1.0;
deltat = 0.1;
eta = 1.0;
omega = 5.0;
N = 2**n;
t0 = 0.1
t = 0.5*N*deltax/deltat;
t5 = t/5;
t20 = t/20
t50 = t/50;

vel1 = np.zeros((N,N,N));
vel2 = np.zeros((N,N,N));
mu,sigma = 0,0.1


C1 = np.random.normal(mu,sigma,size = (N,N,N))
C2 = np.random.normal(mu,sigma,size = (N,N,N))

print(t)

start = time.time()

plotting(N,t0,t5,C1,C2,vel1,vel2,omega,eta,deltax,deltat)

end = time.time()

print(end - start)

start = time.time()

plotting_m(N,t0,t5,C1,C2,vel1,vel2,omega,eta,deltax,deltat)

end = time.time()

print(end - start)


640.0
501.87560296058655
447.62487745285034


In [82]:
def adad(andy,jake):
    andy += jake 
    
andy = np.zeros((3,3))
jake = np.ones((3,3))

adad(andy,jake)
andy
    

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])

In [103]:
def adad(andy,jake):
    andy = andy + jake 
    
andy = np.zeros((3,3))
jake = np.ones((3,3))

adad(andy,jake)
andy
    

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])